In [1]:
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import describe
import datetime
from datetime import timedelta as timedelta
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import signal
import os, sys
sys.path.append( '/home/ugrads/c/clearloveyanzhen/CufflessBP/bioz_processing' )
import sicong_util as su
import sicong_duck_path as sp
import sicong_duck_path as sdp
plt.rcParams.update({'font.size': 13})
import seaborn as sns
import scipy as sp # for signal processing
from scipy import signal
from scipy.spatial import distance
import IPython.display as ipd
import random
import math
import mat73

In [101]:
morph_dict = su.export_morph_feat_to_dict()[0]
finapres_dict = su.export_finapres_to_dict()[0]
# ml = morph_dict['A']['01']['11_29_43'][0]
# ml_df=pd.read_csv(ml)
# fprs = finapres_dict['A']['01']['11_29_43'][0]
# fprs_obj=pd.read_csv(fprs)

In [93]:
fprs_obj.head(10)

,reSYS,fiSYS,reDIA,fiDIA,time (synced to ppg)
0,115.859985,107.482910,74.096680,56.396484,0.502095
1,118.743896,111.007690,73.196411,55.648804,1.341995
2,122.634888,114.501953,74.996948,57.357788,2.176995
3,123.718262,115.951538,74.432373,56.488037,3.046895
4,122.985840,115.509033,73.867798,56.152344,3.936795
5,121.994019,113.952637,73.577881,55.831909,4.811695
6,121.078491,113.250732,73.059082,55.603027,5.701695
7,121.032715,112.289429,75.607300,57.830811,6.571595
8,119.705200,111.816406,75.332642,57.250977,7.421495
9,118.728638,110.168457,77.072144,58.624268,8.256395


In [75]:
ml_df['t_dv'].head(7)

0    [1.849503005010057, 1.6923313698671099, 1.8541...
1    [2.5789680823609076, 2.5016279061085647, 2.583...
2    [3.335108187741769, 3.284411730470118, 3.34017...
3    [4.097917073307517, 4.060550797271277, 4.10301...
4    [4.863393472936506, 4.834029302940828, 4.86850...
5    [5.630203629725889, 5.606176789283891, 5.63531...
6    [6.397775933240138, 6.377563421871572, 6.40289...
Name: t_dv, dtype: object

In [82]:
# # def add_inter_beat(df, cname):
# ptt_list = ml_df['ptt_mst'].tolist()
# ptt_1 = ptt_list[2:]-ptt_list[1:]
# ptt_2 = ptt_list[2:]-ptt_list[1:]
fprs_time = fprs_obj['time (synced to ppg)'].to_numpy()
bioz_ms_t = float(e['t_dv'][1:e['t_dv'].find(',')])
np.where(fprs_time == fprs_time[(fprs_time<=bioz_ms_t)][-1])[0][0]

(array([309]),)

In [108]:
# additional features: changes in ptt, ibi, td
def make_batch(ml_df, fprs_obj, out_x=[], out_y=[]):
    fprs_sys = fprs_obj['reSYS'].to_numpy()
    fprs_dia = fprs_obj['reDIA'].to_numpy()
    fprs_time = fprs_obj['time (synced to ppg)'].to_numpy()
    for i, e in ml_df[2:].iterrows(): 
        feat_list = ml_df.loc[i-2][:-3].tolist()+ml_df.loc[i-1][:-3].tolist()+ml_df.loc[i][:-3].tolist()
        bioz_ms_t = float(e['t_dv'][1:e['t_dv'].find(',')])
        fprs_idx = np.where(fprs_time == fprs_time[(fprs_time<=bioz_ms_t)][-1])[0][0]
        out_x.append(feat_list)
        out_y.append([fprs_sys[fprs_idx], fprs_dia[fprs_idx]])
    return out_x, out_y
out_x=[]; out_y=[]
out_x, out_y = make_batch(ml_df, fprs_obj, out_x=out_x, out_y=out_y)
out_x, out_y = make_batch(ml_df, fprs_obj, out_x=out_x, out_y=out_y)
np.array(out_x).shape, np.array(out_y).shape

((610, 69), (610, 2))

In [112]:
for pname in su.subject_list[:-2]:
    out_x=[]
    out_y=[]
    for session in finapres_dict[pname]:
        print(f'Processing Subject {pname}; Session {session}')
        for trial in finapres_dict[pname][session]:
            for i, file in enumerate(finapres_dict[pname][session][trial]):
                ml_df=pd.read_csv(morph_dict[pname][session][trial][i])
                fprs_obj=pd.read_csv(finapres_dict[pname][session][trial][i])
                out_x, out_y = make_batch(ml_df, fprs_obj, out_x=out_x, out_y=out_y)
    su.email_func(subject=f'Finished Morph Extraction', message=f'Finished Subject {pname}')
    np.save(f'X_file 75{pname} Morph', np.array(out_x))
    np.save(f'y_file 75{pname} Morph', np.array(out_y))
su.email_func(subject=f'Morph Extraction ALL Done')

Processing Subject A; Session 01
Processing Subject A; Session 02
Processing Subject A; Session 03
Processing Subject A; Session 04
Processing Subject A; Session 05
Processing Subject A; Session 06
Processing Subject A; Session 07
Subject: Finished Morph Extraction 

Finished Subject A
Processing Subject C; Session 01
Processing Subject C; Session 02
Processing Subject C; Session 03
Processing Subject C; Session 04
Processing Subject C; Session 05
Processing Subject C; Session 06
Processing Subject C; Session 07
Subject: Finished Morph Extraction 

Finished Subject C
Processing Subject B; Session 01
Processing Subject B; Session 02
Processing Subject B; Session 03
Processing Subject B; Session 04
Processing Subject B; Session 05
Processing Subject B; Session 06
Processing Subject B; Session 07
Subject: Finished Morph Extraction 

Finished Subject B
Processing Subject E; Session 01
Processing Subject E; Session 02
Processing Subject E; Session 03
Processing Subject E; Session 04
Process

IndexError: list index out of range